In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd

csv_path = "/content/drive/MyDrive/Collab_Business_Excell/customer_shopping_behavior.csv"
df = pd.read_csv(csv_path)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df = df.rename(columns={'purchase_amount_(usd)': 'purchase_amount'})


In [ ]:
df.info()
df.describe(include='all')
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount         3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3863 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used         3900 non-null   

,0
customer_id,0
age,0
gender,0
item_purchased,0
category,0
purchase_amount,0
location,0
size,0
color,0
season,0


In [ ]:
df['review_rating'] = df.groupby('category')['review_rating']\
                         .transform(lambda x: x.fillna(x.median()))


In [4]:
# Make sure 'Category' exists
if 'Category' in df.columns and 'Review Rating' in df.columns:
    df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(
        lambda x: x.fillna(x.median())
    )
else:
    print("Category or Review Rating column missing!")

In [7]:
print(df.head())

   Customer ID  Age Gender Item Purchased  Category  Purchase Amount (USD)  \
0            1   55   Male         Blouse  Clothing                     53   
1            2   19   Male        Sweater  Clothing                     64   
2            3   50   Male          Jeans  Clothing                     73   
3            4   21   Male        Sandals  Footwear                     90   
4            5   45   Male         Blouse  Clothing                     49   

        Location Size      Color  Season  Review Rating Subscription Status  \
0       Kentucky    L       Gray  Winter            3.1                 Yes   
1          Maine    L     Maroon  Winter            3.1                 Yes   
2  Massachusetts    S     Maroon  Spring            3.1                 Yes   
3   Rhode Island    M     Maroon  Spring            3.5                 Yes   
4         Oregon    M  Turquoise  Spring            2.7                 Yes   

   Shipping Type Discount Applied Promo Code Used  Previ

In [8]:
#create a column  age_group
labels = ['Young Adult','Adult','Middle-aged','Senior']

if 'age_group' not in df.columns:
    df['age_group'] = pd.qcut(df['Age'], q=4, labels=labels)

In [ ]:
#create a col purchase_frequency_days

frequency_mapping = {
    'weekly': 7,
    'bi-weekly': 14,
    'fortnightly': 14,
    'monthly': 30,
    'quarterly': 90,
    'every 3 months': 90,
    'annually': 365
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)


In [ ]:
#remove promo_code_used becaus it is all equal to purchase_amount
df.columns

In [ ]:
import sqlite3

conn = sqlite3.connect(
    "/content/drive/MyDrive/Collab_Business_Excell/analysis_db.db"
)


In [ ]:
df.to_sql("customers", conn, if_exists="replace", index=False)


3900

In [ ]:
pd.read_sql("SELECT COUNT(*) AS total_rows FROM customers;", conn)


,total_rows
0,3900


In [ ]:
#Total Revenue generated by male vs female customer
pd.read_sql("""
SELECT gender, SUM(purchase_amount) AS total_revenue
FROM customers
GROUP BY gender;
""", conn)

,gender,total_revenue
0,Female,75191
1,Male,157890


In [ ]:
# Customer used a discount but still spent more than the average purchase amount
pd.read_sql("""
SELECT customer_id, purchase_amount, discount_applied
FROM customers
WHERE discount_applied = 'Yes'
AND purchase_amount > (
    SELECT AVG(purchase_amount)
    FROM customers
);
""", conn)

,customer_id,purchase_amount,discount_applied
0,2,64,Yes
1,3,73,Yes
2,4,90,Yes
3,7,85,Yes
4,9,97,Yes
...,...,...,...
834,1667,64,Yes
835,1671,73,Yes
836,1673,73,Yes
837,1674,62,Yes


In [ ]:
#Which are the top 5 products with the highest avrg review rating
pd.read_sql("""
SELECT
    item_purchased,
    ROUND(AVG(review_rating), 2) AS avg_rating
FROM customers
GROUP BY item_purchased
ORDER BY avg_rating DESC
LIMIT 5;
""", conn)

,item_purchased,avg_rating
0,Gloves,3.86
1,Sandals,3.84
2,Boots,3.82
3,Hat,3.80
4,T-shirt,3.78


In [ ]:
#Compare the avrg purcahse Amounts between Standard and Express
pd.read_sql("""
SELECT
    shipping_type,
    ROUND(AVG(purchase_amount), 2) AS avg_purchase_amount
FROM customers
WHERE shipping_type IN ('Standard', 'Express')
GROUP BY shipping_type;
""", conn)


,shipping_type,avg_purchase_amount
0,Express,60.48
1,Standard,58.46


In [ ]:
# DO subscribed customer spen more ? compare spend and total revenue
pd.read_sql("""
SELECT
    COUNT (DISTINCT customer_id) AS total_customers,
    subscription_status,
    ROUND(AVG(purchase_amount),2) as avg_spend,
    ROUND(SUM(purchase_amount),2) as total_revenue
FROM customers
GROUP BY subscription_status;
""", conn)

,total_customers,subscription_status,avg_spend,total_revenue
0,2847,No,59.87,170436.0
1,1053,Yes,59.49,62645.0


In [ ]:
# Which 5 products have the highest percentage of purchase with discounts appiled?
pd.read_sql("""
SELECT
    item_purchased,
    ROUND(
        AVG(CASE
            WHEN discount_applied = 'Yes' THEN 1.0
            ELSE 0.0
        END) * 100, 2
    ) AS discount_percentage
FROM customers
GROUP BY item_purchased
ORDER BY discount_percentage DESC
LIMIT 5;
""", conn)

,item_purchased,discount_percentage
0,Hat,50.00
1,Sneakers,49.66
2,Coat,49.07
3,Sweater,48.17
4,Pants,47.37


In [ ]:
# Segement customers into New,Returning , and Loyal Based on their total number of previous purchases,and show the count of each segment
pd.read_sql("""
SELECT
    CASE
        WHEN previous_purchases = 0 THEN 'New'
        WHEN previous_purchases BETWEEN 1 AND 5 THEN 'Returning'
        ELSE 'Loyal'
    END AS customer_segment,
    COUNT(*) AS customer_count
FROM customers
GROUP BY customer_segment;
""", conn)


,customer_segment,customer_count
0,Loyal,3476
1,Returning,424


In [ ]:
# What are the top 3 most purchased products with in each category
pd.read_sql("""
SELECT
    category,
    item_purchased,
    purchase_count
FROM (
    SELECT
        category,
        item_purchased,
        COUNT(*) AS purchase_count,
        ROW_NUMBER() OVER (
            PARTITION BY category
            ORDER BY COUNT(*) DESC
        ) AS rank_in_category
    FROM customers
    GROUP BY category, item_purchased
)
WHERE rank_in_category <= 3
ORDER BY category, purchase_count DESC;
""",conn)

,category,item_purchased,purchase_count
0,Accessories,Jewelry,171
1,Accessories,Sunglasses,161
2,Accessories,Belt,161
3,Clothing,Pants,171
4,Clothing,Blouse,171
5,Clothing,Shirt,169
6,Footwear,Sandals,160
7,Footwear,Shoes,150
8,Footwear,Sneakers,145
9,Outerwear,Jacket,163


In [ ]:
# Are customers who are repeat buyers (more than 5 previus purchase) also likely to subscribe?
pd.read_sql("""
SELECT
    CASE
        WHEN previous_purchases > 5 THEN 'Repeat Buyer (>5)'
        ELSE 'Non-Repeat Buyer (≤5)'
    END AS customer_type,
    subscription_status,
    COUNT(*) AS customer_count
FROM customers
GROUP BY customer_type, subscription_status
ORDER BY customer_type, subscription_status;
""",conn)

,customer_type,subscription_status,customer_count
0,Non-Repeat Buyer (≤5),No,329
1,Non-Repeat Buyer (≤5),Yes,95
2,Repeat Buyer (>5),No,2518
3,Repeat Buyer (>5),Yes,958


In [ ]:
# What is the revenue contribution of each group
pd.read_sql("""
SELECT
    CASE
        WHEN previous_purchases > 5 THEN 'Repeat Buyer (>5)'
        ELSE 'Non-Repeat Buyer (≤5)'
    END AS customer_type,
    ROUND(SUM(purchase_amount), 2) AS total_revenue,
    ROUND(AVG(purchase_amount), 2) AS avg_purchase_amount
FROM customers
GROUP BY customer_type;
""",conn)

,customer_type,total_revenue,avg_purchase_amount
0,Non-Repeat Buyer (≤5),25692.0,60.59
1,Repeat Buyer (>5),207389.0,59.66


In [ ]:
df.to_csv("customers_final.csv", index=False)
print("CSV exported successfully")


CSV exported successfully
